<a href="https://colab.research.google.com/github/heinsense2/AIO_CaseStudy/blob/main/notebooks/Training_on_FathomNet_Custom_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom training using YOLOv5 on Fathomnet custom dataset

This notebook explains how to train a custom dataset using YOLOv5 to recognize different marine species presnt in the Monterey bay. This notebook serves as a guideline to produce the results presented in the paper

 *Demystifying image-based machine learning: a practical guide to automated analysis of imagery using modern machine learning tools*, 


The data is prepared using code available [here](https://github.com/heinsense2/AIO_CaseStudy/tree/main/data/scripts).

NOTE: If you wish to use this notebook, you will need to make changes to refer to the datast locations and python script parameters amoung others.

Here are the relevant steps:

*   Create the dataset and annotations (labels). Organize directories.
*   Export dataset to YOLOv5
*   Train YOLOv5 to recognize the objects (marine animals) in our dataset
*   Evaluate our YOLOv5 model's performance
*   Run inference to view the model at work


# 1. Install requirements

In [ ]:
# Clone YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install torch==1.8.1 torchvision==0.9.1

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


# 2. Assemble Dataset

To train our model, we need to assemble a dataset of representative images with bounding boxes around the objects we want to detect. Our dataset must be in YOLOv5 format.

The Fathomnet data is downloaded and prepared using code available [here](https://github.com/heinsense2/AIO_CaseStudy/tree/main/data/scripts).


When usig Google Colab, it is recommended to have the data available on Google Drive. So we need to first mount our Google Drive.


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
# List the directory where the data resides
# For example:
#!ls "/content/gdrive/My Drive/data"

# 3. Train Our Custom YOLOv5 model

We are able to pass a number of arguments to ```train.py```. To see all the settable arguments:

>>
```python
python train.py -h
```

Here is what we used:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. We use 300. (Note: typically 300-1000 are used although 1000+ are common here!)
- **data:** Our dataset locaiton is saved in the `data.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the pretrained model yolov5s, which is the small and quickest one.
- **cache:** cache images for faster training

The commad to train in general will have the form:

>>
```python
python train.py --img 640 --batch 16 --epochs 300 --data {data.directory}/{domain}.yaml --weights yolov5s.pt --cache
```


Below is an example on how to train YOLOv5 on data from years prior to 2012 using pretrained ```python
--weights yolov5s.pt```. Image size is set to 640 and we use batch size of 16.


In [ ]:
# Train using pre-2012 data
!python train.py --img 640 --batch 16 --epochs 300 --data  ../data/pre_2012/yolov5/pre_2012.yaml --weights yolov5s.pt --cache

To train on the other spatial/depth and temporal regions using the command above, you only need to change the ```--data``` parameter to point to the corresponding yaml file.

In this notebook, we will continue using pre-2012 data as example code.

# Evaluate Custom YOLOv5 Detector Performance
All results are logged by default to runs/train, with a new experiment directory created for each new training (runs/train/exp2, runs/train/exp3)

Training losses and performance metrics are saved to Tensorboard and also to a CSV logfile results.csv

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs/train/exp*"
%load_ext tensorboard
%tensorboard --logdir runs

You can also validate the trained detection model on the test or out-of-domain datasets by using the val.py script in YOLOv5 and the corresponding yaml file.

For the pre-2012 trained model, the following commands will validate it on the test or out-of-domain datasets, assuming the results were saved to ```runs/train/exp```.

In [ ]:
# Validate model on test dataset
!python val.py --data ../data/pre_2012/yolov5/pre_2012.yaml --weights runs/train/exp/weights/best.pt --task test

# Validate model on out-of-domain (post-2012) dataset
!python val.py --data ../data/post_2012/yolov5/post_2012_as_out_of_domain.yaml --weights runs/train/exp/weights/best.pt --task test



#Run Inference  With Trained Weights on New Images

Once the model is trained, you can run inference witht the best pretrained checkpoint. `best.pt`  on test (`images/test`) or out-of-domain (`all/images`) images.

`--conf 0.65` only detections where the confidence level of it being an actual detection is 0.65 or greater.


In [ ]:
# Run detection on pre-2012 images
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.65 --source "../data/pre_2012/yolov5/images/test"

# Run detection on out-of-domain (post-2012) images
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.65 --source "../data/post_2012/yolov5/all/images"



To visualize the detection results, there are many options. One simple way using IPython display is to presented below.

In [ ]:
# Display inference on ALL resulting detection images

import glob
from IPython.display import Image, display

# Assume resulting images from detection are in runs/detect/exp
for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.png'): #assuming PNG
    display(Image(filename=imageName))
    print("\n")

# Export Trained Weights for Future Inference
You can now export the trained weights from our detector for inference on your device elsewhere.


In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')